In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date


from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier


In [2]:
# data files
sample_file = "../data/queue_dataset_train_small_sample.csv"
train_file = "../data/queue_dataset_train.csv"
test_file = "../data/queue_dataset_test.csv"
baseline_res = "../data/baseline_submission.csv"

In [3]:
# read train sample
sample_data = pd.read_csv(sample_file)
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)
train_data.head()

,id,date,branch_name,customer_gender,customer_age_appl,customer_city,service_name_organization,service_name,service_name_2,time_start_process,operator_count,previous_customer_count,service_canceled
0,0,2019-07-04,Masallı ASAN,F,36-40,OTHERS,Daxili İşlər Nazirliyi,Ümumvətəndaş pasportlarının verilməsi və dəyiş...,Ümumvətəndaş pasportlarının verilməsi və dəyiş...,11:27:32.0000000,5,0.0,0
1,1,2019-04-17,Masallı ASAN,M,16-20,OTHERS,Vergilər Nazirliyi,Kommersiya hüquqi şəxslərin və vergi ödəyicilə...,Kommersiya hüquqi şəxslərin və vergi ödəyicilə...,11:44:04.0000000,3,0.0,0
2,2,2019-07-10,Masallı ASAN,M,36-40,BAKI,Ədliyyə Nazirliyi,Etibarnamə,Notariat fəaliyyəti,12:14:28.0000000,4,0.0,0
3,3,2019-08-16,Masallı ASAN,F,26-30,BAKI,Daxili İşlər Nazirliyi,Ümumvətəndaş pasportlarının verilməsi və dəyiş...,Ümumvətəndaş pasportlarının verilməsi və dəyiş...,14:45:34.0000000,5,1.0,0
4,4,2019-09-24,Masallı ASAN,F,66-70,OTHERS,Ədliyyə Nazirliyi,İmzanın təsdiqi,Notariat fəaliyyəti,15:41:37.0000000,5,3.0,0


In [4]:
train_data.tail()

,id,date,branch_name,customer_gender,customer_age_appl,customer_city,service_name_organization,service_name,service_name_2,time_start_process,operator_count,previous_customer_count,service_canceled
4510019,4510019,2018-11-04,Sabirabad ASAN,F,6-10,KÜRDƏMİR,Funksional Yardımçı xidmətlər,"Tibbi arayışların verilməsi (qan qrupu, boyun ...",Tibb xidmətləri,11:38:50.0000000,1,0.0,0
4510020,4510020,2018-08-16,Sabirabad ASAN,M,36-40,OTHERS,Daxili İşlər Nazirliyi,Ümumvətəndaş pasportlarının verilməsi və dəyiş...,Ümumvətəndaş pasportlarının verilməsi və dəyiş...,09:31:31.0000000,2,0.0,0
4510021,4510021,2019-04-18,Sabirabad ASAN,F,0-5,KÜRDƏMİR,Daxili İşlər Nazirliyi,Şəxsiyyət vəsiqələrinin verilməsi və dəyişdiri...,Şəxsiyyət vəsiqəsi və arayışların verilməsi,14:22:02.0000000,5,1.0,0
4510022,4510022,2018-09-07,Sabirabad ASAN,F,NaN,KÜRDƏMİR,Funksional Yardımçı xidmətlər,"Tibbi arayışların verilməsi (qan qrupu, boyun ...",Tibb xidmətləri,10:14:28.0000000,1,1.0,0
4510023,4510023,2019-07-17,Sabirabad ASAN,F,0-5,KÜRDƏMİR,Daxili İşlər Nazirliyi,Şəxsiyyət vəsiqələrinin verilməsi və dəyişdiri...,Şəxsiyyət vəsiqəsi və arayışların verilməsi,10:37:08.0000000,4,10.0,0


In [5]:
# sample_data[["customer_age_appl"]].hist(bins=20)
train_data.describe(percentiles=[0.25, 0.75])

,id,operator_count,previous_customer_count,service_canceled
count,4.510024e+06,4.510024e+06,4.510024e+06,4.510024e+06
mean,2.255012e+06,4.251089e+00,5.088331e+00,7.959581e-02
std,1.301932e+06,2.359285e+00,8.535608e+00,2.706665e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.127506e+06,2.000000e+00,0.000000e+00,0.000000e+00
50%,2.255012e+06,4.000000e+00,2.000000e+00,0.000000e+00
75%,3.382517e+06,6.000000e+00,6.000000e+00,0.000000e+00
max,4.510023e+06,1.200000e+01,1.430000e+02,1.000000e+00


In [6]:
# Statistics between "prev_customer" and "cancelled"
train_data[['previous_customer_count', 'service_canceled']].groupby(['previous_customer_count'], as_index=False).mean().sort_values(by='service_canceled', ascending=False)


,previous_customer_count,service_canceled
109,109.0,0.600000
105,105.0,0.583333
104,104.0,0.533333
103,103.0,0.500000
99,99.0,0.481481
...,...,...
123,123.0,0.000000
124,124.0,0.000000
125,125.0,0.000000
112,112.0,0.000000


In [7]:
# Statistics between "prev_customer" and "cancelled"
# sample_data[['previous_customer_count', 'service_canceled']].groupby(['previous_customer_count'], as_index=False).mean().sort_values(by='service_canceled', ascending=False)
train_data['branch_name'].value_counts()

3 saylı ASAN      856317
1 saylı ASAN      762272
5 saylı ASAN      721930
4 saylı ASAN      595069
Sumqayıt ASAN     462962
Gəncə ASAN        372058
2 saylı ASAN      370168
Bərdə ASAN         97750
Sabirabad ASAN     90661
Quba ASAN          84947
Masallı ASAN       62081
Qəbələ ASAN        33809
Name: branch_name, dtype: int64

In [8]:
# sample_data["customer_age_appl"].unique()
train_data[['branch_name', 'service_canceled']].groupby(['branch_name']).mean().sort_values(by='service_canceled', ascending=True)

,service_canceled
branch_name,
Qəbələ ASAN,0.037564
Sabirabad ASAN,0.046249
Bərdə ASAN,0.047233
Sumqayıt ASAN,0.057063
Quba ASAN,0.059225
1 saylı ASAN,0.072953
Masallı ASAN,0.075498
5 saylı ASAN,0.080087
3 saylı ASAN,0.086812


In [9]:
train_data.isnull().sum()

id                               0
date                             0
branch_name                      0
customer_gender                  0
customer_age_appl            32616
customer_city                    0
service_name_organization        0
service_name                     0
service_name_2                   0
time_start_process           29213
operator_count                   0
previous_customer_count          0
service_canceled                 0
dtype: int64

In [10]:
test_data.isnull().sum()

id                              0
date                            0
branch_name                     0
customer_gender                 0
customer_age_appl            1721
customer_city                   0
service_name_organization       0
service_name                    0
service_name_2                  0
time_start_process           2409
operator_count                  0
previous_customer_count         0
dtype: int64

# Preprocessing ideas:

    1. convert dates to one-hot vector that indicates weekdays  
    2. convert ages to one-hot encoding. group by for different age groups  
        such as over 60 (teqaudcu), under 20 (teens), 20-30 (youngs), 30-50 (working class)
        50-60 (oldish)  
    3. convert service name, branch name and service organization to one-hot encoding  
    4. fill missing values with average or remove them  
    5. cancellation is much less than approved requests, apply data balancing to train better  
    6. remove operator count, unnecessary data  
    7. convert times to one-hot vector according to the time of the day
        9-12 : early day, 12-3 mid day, 3-6 late  
    8. Remove gender and customer city (hopefully no bias in those matters)  
    9. Previous customer count seems necessary, so we may divide it up into few clusters and 
        apply one-hot encoding: 0-5 few people, 5-20 meh, 20+ needs 1 hour (maybe we can apply
        deterministic approach for such cases)  
    10. We can weight services based on time required to complete.  

In [11]:
# utility functions
branches = ['Masallı ASAN', '1 saylı ASAN', '4 saylı ASAN', 'Qəbələ ASAN', '2 saylı ASAN', 'Sumqayıt ASAN']
service_org = ['Daxili İşlər Nazirliyi', 'Vergilər Nazirliyi',
               'Ədliyyə Nazirliyi', 'Funksional Yardımçı xidmətlər',
               'Azərbaycan Respublikasının Dövlət Sosial Müdafiə Fondu',
               'Əmlak Məsələləri Dövlət Komitəsi', 
               'Dövlət Miqrasiya Xidməti']
service_names = ['Ümumvətəndaş pasportlarının verilməsi və dəyişdirilməsi',
       'Kommersiya hüquqi şəxslərin və vergi ödəyicilərinin qeydiyyatı',
       'Etibarnamə', 'İmzanın təsdiqi', 'Digər xidmətlər',
       'Yaşayış yeri üzrə qeydiyyatda olan şəxslər haqqında arayışın verilməsi',
       'Azərcell', 'Vərəsəlik', 'Foto',
       'Əmək pensiyalarinin təyin edilməsi', 'Təkrar çıxarış - mənzil',
       'Asan İmza sertifikatının verilməsi',
       'Şəxsiyyət vəsiqələrinin verilməsi və dəyişdirilməsi',
       'Tibbi arayışların verilməsi (qan qrupu, boyun ölçülməsi, göz rəngi)',
       'Ölümün qeydə alınması, övladlığa götürmənin qeydə alınması, atalığın müəyyən edilməsinin qeydə alınması, nikahın pozulmasının qeydə alınması, adın, ata adının və soyadın dəyişdirilməsinin qeydə alınması, vətəndaşlıq vəziyyəti aktlarının dövlət qeydiyyatı haqqında şəhadətnamələrin (təkrar şəhadətnamələrin) verilməsi',
       'Surətin təsdiqi',
       'Əcnəbinin və ya vətəndaşlığı olmayan şəxsin olduğu yer üzrə qeydiyyata alınması',
       'İlkin qeydiyyat', 'Sürücülük vəsiqələrinin dəyişdirilməsi',
       'Məhkumluq barədə arayışların verilməsi', 'Miqrasiya xidməti',
       'İlkin çıxarış - mənzil', 'Tərcümə', 'Kapital Bank']




def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 


def restore_time(sample_data):
    # restore nan times based on age, gender and service type
    # first we need to change all ages to numerical values (avg of interval) to work 
    def process(q_time):
        q_time = list(map(lambda x: int(x[:2]), q_time.split(":")))
        return 60*q_time[0] + q_time[1]

    sample_copy = sample_data.copy()
    sample_copy['time_start_process'] = sample_copy['time_start_process'].map(lambda q_time : q_time if pd.isnull(q_time) else process(q_time))

    grouped_data = sample_copy.groupby(['customer_gender','service_name', 'customer_age_appl'])
    grouped_data_mean = grouped_data.mean()
    grouped_data_mean = grouped_data_mean.reset_index()[['customer_gender', 'service_name', 'customer_age_appl', 'time_start_process']]
    
    def missing_time(entry):
        condition = (
                (grouped_data_mean['customer_gender'] == entry['customer_gender']) & 
                (grouped_data_mean['service_name'] == entry['service_name']) &
                (grouped_data_mean['customer_age_appl'] == entry['customer_age_appl'])
                ) 
        q_time = int(grouped_data_mean[condition]['time_start_process'].values[0])
        return str(q_time // 60) + ":" + str(q_time % 60) + ":00"
        
    sample_data['time_start_process'] = sample_data.apply(lambda entry: missing_time(entry) if pd.isnull(entry['time_start_process']) else entry['time_start_process'],axis=1)
    return sample_data


def restore_age(sample_data):
    # restore time based on age, gender, service type
    sample_copy = sample_data.copy()
    sample_copy['customer_age_appl'] = sample_copy['customer_age_appl'].map(lambda age : age if pd.isnull(age) else list(map(int, age.split("-")))[0] + 2)

    grouped_data = sample_copy.groupby(['customer_gender','service_name'])
    grouped_data_mean = grouped_data.mean()
    grouped_data_mean = grouped_data_mean.reset_index()[['customer_gender', 'service_name', 'customer_age_appl']]
    
    def missing_age(entry):
        condition = (
                (grouped_data_mean['customer_gender'] == entry['customer_gender']) & 
                (grouped_data_mean['service_name'] == entry['service_name'])
                ) 
        age = int(grouped_data_mean[condition]['customer_age_appl'].values[0])
        rem = age % 5
        return str(age - rem + 1) + "-" + str(age - rem + 5)
        
    sample_data['customer_age_appl'] = sample_data.apply(lambda entry: missing_age(entry) if pd.isnull(entry['customer_age_appl']) else entry['customer_age_appl'],axis=1)
    return sample_data


def categorize_dates(sample_data):
    def weekdays(inp_date):
        inp_date = list(map(int, inp_date.split("-")))
        return date(inp_date[0], inp_date[1], inp_date[2]).weekday()

    sample_data['mon'] = sample_data['date'].map(lambda inp_date : 1 if weekdays(inp_date)==0 else 0)
    sample_data['tue'] = sample_data['date'].map(lambda inp_date : 1 if weekdays(inp_date)==1 else 0)
    sample_data['wed'] = sample_data['date'].map(lambda inp_date : 1 if weekdays(inp_date)==2 else 0)
    sample_data['thr'] = sample_data['date'].map(lambda inp_date : 1 if weekdays(inp_date)==3 else 0)
    sample_data['fri'] = sample_data['date'].map(lambda inp_date : 1 if weekdays(inp_date)==4 else 0)
    sample_data['sat'] = sample_data['date'].map(lambda inp_date : 1 if weekdays(inp_date)==5 else 0)
    sample_data['sun'] = sample_data['date'].map(lambda inp_date : 1 if weekdays(inp_date)==6 else 0)
    return sample_data
    

def categorize_ages(sample_data):
    def process(age):
        return list(map(int, age.split("-")))
    
    sample_data['kid_age'] = sample_data['customer_age_appl'].map(lambda age : 1 if process(age)[1]<=15 else 0)
    sample_data['young_age'] = sample_data['customer_age_appl'].map(lambda age : 1 if (process(age)[1]<=30 and process(age)[0]>15) else 0)
    sample_data['mid_age'] = sample_data['customer_age_appl'].map(lambda age : 1 if (process(age)[1]<=55 and process(age)[0]>30) else 0)
    sample_data['old_age'] = sample_data['customer_age_appl'].map(lambda age : 1 if process(age)[0]>55 else 0)
    return sample_data
    

def categorize_time(sample_data):
    def process(time):
         return list(map(lambda x: int(x[:2]), time.split(":")))
    
    sample_data['early_hr'] = sample_data['time_start_process'].map(lambda time : 1 if process(time)[1]<=11 else 0)
    sample_data['mid_hr'] = sample_data['time_start_process'].map(lambda time : 1 if (process(time)[1]<=14 and process(time)[0]>11) else 0)
    sample_data['late_hr'] = sample_data['time_start_process'].map(lambda time : 1 if process(time)[0]>14 else 0)
    return sample_data
    
                                                               
def categorize_queue(sample_data):
    sample_data['sm_queue'] = sample_data['previous_customer_count'].map(lambda queue : 1 if queue<=5 else 0)
    sample_data['mid_queue'] = sample_data['previous_customer_count'].map(lambda queue : 1 if (5<queue<=20) else 0)
    sample_data['big_queue'] = sample_data['previous_customer_count'].map(lambda queue : 1 if queue>20 else 0)
    return sample_data


def remove_unused_cols(sample_data):
    sample_data.drop(['customer_age_appl'], axis=1, inplace=True)
    sample_data.drop(['customer_gender'], axis=1, inplace=True)
    sample_data.drop(['time_start_process'], axis=1, inplace=True)
    sample_data.drop(['operator_count'], axis=1, inplace=True)
    sample_data.drop(['date'], axis=1, inplace=True)
    sample_data.drop(['customer_city'], axis=1, inplace=True)
    sample_data.drop(['service_name_2'], axis=1, inplace=True)
    sample_data.drop(['previous_customer_count'], axis=1, inplace=True)
    return sample_data
                                                 
                                                 

In [12]:
train_data = restore_age(train_data)
train_data = restore_time(train_data)
train_data.isnull().sum()

id                           0
date                         0
branch_name                  0
customer_gender              0
customer_age_appl            0
customer_city                0
service_name_organization    0
service_name                 0
service_name_2               0
time_start_process           0
operator_count               0
previous_customer_count      0
service_canceled             0
dtype: int64

In [13]:
test_data = restore_age(test_data)
test_data = restore_time(test_data)
test_data.isnull().sum()

id                           0
date                         0
branch_name                  0
customer_gender              0
customer_age_appl            0
customer_city                0
service_name_organization    0
service_name                 0
service_name_2               0
time_start_process           0
operator_count               0
previous_customer_count      0
dtype: int64

In [ ]:
train_data = encode_and_bind(train_data, "service_name_organization")
train_data = encode_and_bind(train_data, "branch_name")
train_data = encode_and_bind(train_data, "service_name")
train_data = categorize_time(train_data)
train_data = categorize_dates(train_data)
train_data = categorize_ages(train_data)
train_data = categorize_queue(train_data)
train_data = remove_unused_cols(train_data)

train_data.head()

In [ ]:
test_data = encode_and_bind(test_data, "service_name_organization")
test_data = encode_and_bind(test_data, "branch_name")
test_data = encode_and_bind(test_data, "service_name")
test_data = categorize_time(test_data)
test_data = categorize_dates(test_data)
test_data = categorize_ages(test_data)
test_data = categorize_queue(test_data)
test_data = remove_unused_cols(test_data)

test.data()

# Classification Ideas

    1. Assuming 1 person needs at least 5 mins on average for the service and if (queue cnt)*5
        is gt end of work hr, then request will be cancelled
    2. Use Random Forest method
    3. Try xgbboost

In [ ]:
# Creating data for training
X_train = train_data.drop(['service_canceled', "id"], axis=1)
Y_train = train_data["service_canceled"]
X_test = test_data.drop(["id"], axis=1)

In [ ]:
# Select algorithms, Use the algorithm you want to use as classifier
classifier = RandomForestClassifier(n_estimators = 200, random_state = 42)

# Check the accuracy, AUC, and ROC curve of the classifier set above
classifier.fit(X_train, Y_train)
accuracy = classifier.score(X_train, Y_train) * 100
Y_train_pred = classifier.predict_proba(X_train)[:, 1]


In [ ]:
FPR, TPR, thresholds = roc_curve(Y_train, Y_train_pred)
AUC = roc_auc_score(Y_train, Y_train_pred)

plt.plot(FPR, TPR)
print("Accuracy: ", "{0:.2f}".format(accuracy))
print("Area Under the Curve: ", "{0:.2f}".format(AUC))


In [ ]:
# Print the test data prediction and generation of kaggle submission file
predict = classifier.predict(X_test)
predict = np.round(predict)


In [ ]:
# Print the test data prediction and generation of kaggle submission file
predict = classifier.predict(X_test)

In [ ]:
# Create kaggle submission file
submission = pd.DataFrame({'id': preprocessing_test_data['id'], 'service_canceled': predict})
submission.to_csv('submission.csv', index=False)

In [ ]:
test_data = pd.read_csv(test_file)
fname = "dummy_submit.csv"
length = train_data.shape[0]
print(length)


In [ ]:
def write_submission(fname):
    with open(fname, mode='w+') as f:
        f.write('id,service_canceled\n') 
        for i in range(364162):
            f.write(f'{i},0') 
            f.write('\n') 

In [ ]:
train_Data['service_name'].replace(service_names, list(range(len(service_names))), inplace=True)
